## DataFrame Validation with Engarde

In this notebook, we'll take a look at how to validate data within `pandas.DataFrame` objects. [Engarde](https://github.com/TomAugspurger/engarde) is a library created by Tom Augspurger, which allows you to write both function decorators or utilize built-in functions to test your DataFrame with specific validation rules or definitions.

In [ ]:
import pandas as pd
import engarde.decorators as ed
from datetime import datetime

In [ ]:
sales = pd.read_csv('data/sales_data_duped_with_nulls.csv')

## Data Quality Check

In [ ]:
sales.head()

In [ ]:
sales.dtypes

### Engarde lets us track datatypes. Lets define what datatypes we expect our data to contain.

In [ ]:
new_dtypes = {
    'timestamp': 'datetime64[ns]',
    'city': object,
    'store_id': int,
    'sale_number': float,
    'sale_amount': float,
    'associate': object
}

#### we will check that our Dataframe (production data for example) contains the data types we expect
#### as well as that it contains exactly 6 columns (None for rows means we don't care how many rows we get)

In [ ]:
@ed.has_dtypes(new_dtypes)
@ed.is_shape((None, 6))
def update_dtypes(sales):
    sales['timestamp'] = sales.timestamp.map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
    
    return sales

In [ ]:
sales = update_dtypes(sales)

In [ ]:
sales.dtypes

### we did not get an exception which means that our requirements were met

## Now we want to remove poor quality data

we remove duplicates and null values and then make sure that no values are missing using none_missing

In [ ]:
@ed.has_dtypes(new_dtypes)
@ed.is_shape((None, 6))
@ed.none_missing()
def remove_poor_quality_data(sales):
    sales = sales.drop_duplicates()
    sales = sales.dropna(subset=['sale_amount', 'store_id', 'sale_number', 
                                 'city', 'associate'])
    return sales

In [ ]:
sales = remove_poor_quality_data(sales)

Again, we get no errors and we can move on

Now, we add some computational columns:

In [ ]:
final_types = new_dtypes.copy()
final_types.update({
    'store_total': float,
    'associate_total': float
})

In [ ]:
@ed.has_dtypes(final_types)
@ed.none_missing()
@ed.unique_index()
@ed.is_shape((None, 8))
def calculate_store_sales(sales):
    sales['store_total'] = sales.groupby('store_id').transform(sum)['sale_amount']
    sales['associate_total'] = sales.groupby('associate').transform(sum)['sale_amount']
    return sales

In [ ]:
sales = calculate_store_sales(sales)

## Lets fix the above error

In [ ]:
@ed.has_dtypes(final_types)
@ed.none_missing()
def calculate_store_sales(sales):
    sales['store_total'] = sales.groupby('store_id').transform(sum)['sale_amount']
    sales['associate_total'] = sales.groupby('associate').transform(sum)['sale_amount']

    sales['store_total'] = pd.to_numeric(sales['store_total'])
    sales['associate_total'] = pd.to_numeric(sales['associate_total'])
    return sales


In [ ]:
sales = calculate_store_sales(sales)

In [ ]:
sales.dtypes

### Great! Error fixed. Now let's add a new constraint:

In [ ]:
@ed.has_dtypes(final_types)
@ed.none_missing()
@ed.within_range({'sale_amount': (-1550, 1550)})
@ed.is_shape((None, 8))
def calculate_store_sales(sales):
    sales['store_total'] = sales.groupby('store_id').transform(sum)['sale_amount']
    sales['associate_total'] = sales.groupby('associate').transform(sum)['sale_amount']

    sales['store_total'] = pd.to_numeric(sales['store_total'])
    sales['associate_total'] = pd.to_numeric(sales['associate_total'])
    return sales


In [ ]:
sales = calculate_store_sales(sales)

In [ ]:
sales.where((sales['sale_amount']>1550) | (sales['sale_amount']<-1550)).dropna()

#### Now we need to ask ourselves:
What is the reason for these errors?
And then => decide how to handle them.